In [11]:
import numpy as np
import pandas as pd
import random

In [12]:
# Parameters
alfa = 0.9
beta = 0.2
gamma = 0.1
lambda0 = 1
lambda1 = 5


In [19]:
def forward_simulation(N=100,T=100):
    
    c_states = [0,1,2]
    
    # T1 C is 1
    
    c_list = [2]
    
    # TO DO: Generate C's
    
    for c in range(N):
        
        previous_c = c_list[-1]
            
        if previous_c == 2:

            c = random.choices(c_states, weights=(beta/2,beta/2,1-beta), k=1)

            c_list.append(c[0])

        elif previous_c == 1:

            c = random.choices(c_states, weights=(0,1-gamma,gamma), k=1)

            c_list.append(c[0])


        elif previous_c == 0:


            c = random.choices(c_states, weights=(1-gamma,0,gamma), k=1)

            c_list.append(c[0])                

    # TO DO: Generate Z
    
    
    
    
    
    
    
    # TO DO: Generate X
    return()
    
    
    
    
    
    
    
    